## Euroleague

#### Played Games

In [8]:
import requests
import json
import xmltodict
import pandas as pd

# The actual endpoint URL for getting game codes is:
BASE_URL = "https://api-live.euroleague.net/v1/results/" 
SEASON_CODE = "E2025" # Your desired season

params = {
    "seasonCode": SEASON_CODE,
}

# --- TEMPORARY FIX: ADD verify=False ---
try:
    response = requests.get(
        BASE_URL, 
        params=params, 
        timeout=60, 
        verify=False  # <--- THIS IS THE FIX
    )
    
    # Optional: Suppress the InsecureRequestWarning if using verify=False
    requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
    
    response.raise_for_status() # Check for other HTTP errors (4xx/5xx)
    
    # Process the data just like the EuroLeagueData class does:
    data = xmltodict.parse(response.content)
    game_codes_df = pd.DataFrame(data["results"]["game"])
    
    # print("Successfully retrieved game codes by skipping SSL verification.")
    # print(game_codes_df.head())
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching data (even with verify=False): {e}")

# If successful, you now have the game codes and can continue the schedule logic manually.
# print(game_codes_df)

# game_codes_df = game_codes_df.sort_values("date")
game_codes_df['gamenumber_numeric'] = pd.to_numeric(
    game_codes_df['gamenumber'], 
    errors='coerce'
)

game_codes_df.sort_values(by='gamenumber_numeric', ascending=True)
game_codes_df

,round,gameday,date,time,gamenumber,gamecode,group,hometeam,homecode,homescore,awayteam,awaycode,awayscore,played,gamenumber_numeric
0,RS,5,"Oct 17, 2025",20:45,47,E2025_47,Regular Season,BASKONIA VITORIA-GASTEIZ,BAS,79,PARTIZAN MOZZART BET BELGRADE,PAR,91,true,47
1,RS,8,"Oct 30, 2025",20:05,73,E2025_73,Regular Season,MACCABI RAPYD TEL AVIV,TEL,92,CRVENA ZVEZDA MERIDIANBET BELGRADE,RED,99,true,73
2,RS,10,"Nov 11, 2025",17:00,96,E2025_96,Regular Season,DUBAI BASKETBALL,DUB,102,CRVENA ZVEZDA MERIDIANBET BELGRADE,RED,86,true,96
3,RS,1,"Sep 30, 2025",20:45,7,E2025_7,Regular Season,VIRTUS BOLOGNA,VIR,74,REAL MADRID,MAD,68,true,7
4,RS,8,"Oct 30, 2025",20:30,72,E2025_72,Regular Season,FC BAYERN MUNICH,MUN,86,VIRTUS BOLOGNA,VIR,70,true,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,RS,1,"Sep 30, 2025",20:30,2,E2025_2,Regular Season,BASKONIA VITORIA-GASTEIZ,BAS,96,OLYMPIACOS PIRAEUS,OLY,102,true,2
106,RS,7,"Oct 28, 2025",20:30,61,E2025_61,Regular Season,BASKONIA VITORIA-GASTEIZ,BAS,92,DUBAI BASKETBALL,DUB,85,true,61
107,RS,9,"Nov 07, 2025",18:30,87,E2025_87,Regular Season,ANADOLU EFES ISTANBUL,IST,93,EA7 EMPORIO ARMANI MILAN,MIL,97,true,87
108,RS,2,"Oct 03, 2025",20:00,17,E2025_17,Regular Season,LDLC ASVEL VILLEURBANNE,ASV,102,BASKONIA VITORIA-GASTEIZ,BAS,95,true,17


#### Future Games

In [6]:
import requests
import xmltodict
import pandas as pd

BASE_URL = "https://api-live.euroleague.net/v1/schedules"
SEASON_CODE = "E2025"

params = {"seasonCode": SEASON_CODE}

response = requests.get(BASE_URL, params=params, timeout=60, verify=False)
response.raise_for_status()

data = xmltodict.parse(response.content)

# Correct structure based on your output:
games = pd.DataFrame(data["schedule"]["item"])

# Normalize data types
games["played"] = games["played"].astype(str).str.lower() == "true"

# Convert date to datetime (EuroLeague uses formats like "Sep 30, 2025")
games["date"] = pd.to_datetime(games["date"], errors="coerce")

# Filter to unplayed games (future games)
future_games = games[~games["played"]].copy()

# Sort chronologically
future_games = future_games.sort_values("date")


future_games

,gameday,round,arenacode,arenaname,arenacapacity,date,startime,endtime,group,game,gamecode,hometeam,homecode,hometv,awayteam,awaycode,awaytv,confirmeddate,confirmedtime,played
110,12,RS,AUZ,LDLC ARENA,12523,2025-11-19,20:00,23:30,Regular Season,111,E2025_111,LDLC ASVEL VILLEURBANNE,ASV,ASV,AS MONACO,MCO,ASM,true,true,False
112,12,RS,AVG,TURKCELL BASKETBALL DEVELOPMENT CENTER,10000,2025-11-20,18:30,22:00,Regular Season,112,E2025_112,ANADOLU EFES ISTANBUL,IST,EFS,FC BARCELONA,BAR,BAR,true,true,False
113,12,RS,AUM,TELEKOM CENTER ATHENS,18500,2025-11-20,20:15,23:45,Regular Season,114,E2025_114,PANATHINAIKOS AKTOR ATHENS,PAN,PAO,DUBAI BASKETBALL,DUB,DUB,true,true,False
117,12,RS,ASX,UNIPOL FORUM,11200,2025-11-20,20:30,00:00,Regular Season,113,E2025_113,EA7 EMPORIO ARMANI MILAN,MIL,EA7,HAPOEL IBI TEL AVIV,HTA,HTA,true,true,False
116,12,RS,ATI,MOVISTAR ARENA,13109,2025-11-20,20:45,00:15,Regular Season,115,E2025_115,REAL MADRID,MAD,RMB,ZALGIRIS KAUNAS,ZAL,ZAL,true,true,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,38,RS,ACA,COCA-COLA ARENA,13221,2026-04-17,18:00,21:30,Regular Season,378,E2025_378,DUBAI BASKETBALL,DUB,DUB,VALENCIA BASKET,PAM,VBC,true,true,False
375,38,RS,AUR,SALLE GASTON MEDECIN,5000,2026-04-17,19:30,23:00,Regular Season,377,E2025_377,AS MONACO,MCO,ASM,HAPOEL IBI TEL AVIV,HTA,HTA,true,true,False
370,38,RS,AMH,PALAU BLAUGRANA,7585,2026-04-17,20:30,00:00,Regular Season,379,E2025_379,FC BARCELONA,BAR,BAR,FC BAYERN MUNICH,MUN,BAY,true,true,False
377,38,RS,ATJ,ZALGIRIO ARENA,15415,2026-04-17,19:00,22:30,Regular Season,380,E2025_380,ZALGIRIS KAUNAS,ZAL,ZAL,PARIS BASKETBALL,PRS,PBB,true,true,False


#### Standings

In [12]:
import requests
import xmltodict
import pandas as pd

BASE_URL = "https://api-live.euroleague.net/v1/standings"
SEASON_CODE = "E2025"

params = {"seasonCode": SEASON_CODE}

response = requests.get(BASE_URL, params=params, timeout=60, verify=False)
response.raise_for_status()

data = xmltodict.parse(response.content)

# Correct root key:
standings_root = data["standings"]

# Group section
g = standings_root["group"]

group_name = g["@name"]
round_name = g["@round"]
game_number = g["@gamenumber"]

teams = g["team"]   # list

rows = []
for t in teams:
    rows.append({
        "group": group_name,
        "round": round_name,
        "gameNumber": int(game_number),
        "teamName": t["name"],
        "teamCode": t["code"],
        "ranking": int(t["ranking"]),
        "totalGames": int(t["totalgames"]),
        "wins": int(t["wins"]),
        "losses": int(t["losses"]),
        "pointsFavour": int(t["ptsfavour"]),
        "pointsAgainst": int(t["ptsagainst"]),
        "difference": int(t["difference"]),
    })

standings_df = pd.DataFrame(rows).sort_values("ranking")
standings_df

,group,round,gameNumber,teamName,teamCode,ranking,totalGames,wins,losses,pointsFavour,pointsAgainst,difference
0,Regular Season,RS,11,Hapoel IBI Tel Aviv,HTA,1,11,8,3,1000,935,65
1,Regular Season,RS,11,Crvena Zvezda Meridianbet Belgrade,RED,2,11,8,3,965,902,63
2,Regular Season,RS,11,Zalgiris Kaunas,ZAL,3,11,7,4,951,876,75
3,Regular Season,RS,11,Olympiacos Piraeus,OLY,4,11,7,4,945,886,59
4,Regular Season,RS,11,Panathinaikos AKTOR Athens,PAN,5,11,7,4,969,959,10
5,Regular Season,RS,11,FC Barcelona,BAR,6,11,7,4,948,947,1
6,Regular Season,RS,11,EA7 Emporio Armani Milan,MIL,7,11,6,5,890,872,18
7,Regular Season,RS,11,Valencia Basket,PAM,8,11,6,5,991,978,13
8,Regular Season,RS,11,AS Monaco,MCO,9,11,6,5,944,931,13
9,Regular Season,RS,11,Fenerbahce Beko Istanbul,ULK,10,11,6,5,870,870,0


#### SOS

In [15]:
import pandas as pd

def compute_strength_of_schedule(standings_df, future_games_df, n_games=None):
    # --- Clean up -------------

    # Ensure date is datetime
    future_games_df["date"] = pd.to_datetime(future_games_df["date"], errors="coerce")

    # Map teamCode → ranking
    rankings = standings_df.set_index("teamCode")["ranking"].to_dict()

    # All EuroLeague team codes in standings
    teams = standings_df["teamCode"].unique()

    results = []

    for team in teams:

        # Team plays as home
        home_games = future_games_df[future_games_df["homecode"] == team].copy()
        home_games["opponent"] = home_games["awaycode"]

        # Team plays as away
        away_games = future_games_df[future_games_df["awaycode"] == team].copy()
        away_games["opponent"] = away_games["homecode"]

        # Combine & sort
        next_games = pd.concat([home_games, away_games]).sort_values("date")

        # Keep only next n games
        next_games = next_games.head(n_games)

        if next_games.empty:
            # Team has no remaining games
            results.append({
                "teamCode": team,
                "teamName": standings_df.loc[standings_df["teamCode"] == team, "teamName"].iloc[0],
                "SOS": None,
                "gamesRemaining": 0,
                "opponents": [],
                "opponentRankings": []
            })
            continue
        
        opp_codes = next_games["opponent"].tolist()
        opp_rankings = [rankings.get(code, None) for code in opp_codes]

        # Compute SOS
        sos = sum(opp_rankings) / len(opp_rankings)

        results.append({
            "teamCode": team,
            "teamName": standings_df.loc[standings_df["teamCode"] == team, "teamName"].iloc[0],
            "SOS": sos,
            "gamesRemaining": len(next_games),
            "opponents": opp_codes,
            "opponentRankings": opp_rankings,
            "gameDates": next_games["date"].tolist(),
            "gameCodes": next_games["gamecode"].tolist()
        })

    sos_df = pd.DataFrame(results).sort_values("SOS")
    return sos_df


#### SOS 5 Games

In [14]:
sos_df = compute_strength_of_schedule(standings_df, future_games, n_games=5)
sos_df

,teamCode,teamName,SOS,gamesRemaining,opponents,opponentRankings,gameDates,gameCodes
11,PRS,Paris Basketball,7.0,5,"[OLY, DUB, MCO, ZAL, BAR]","[4, 13, 9, 3, 6]","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_117, E2025_121, E2025_131, E2025_144, E..."
1,RED,Crvena Zvezda Meridianbet Belgrade,7.2,5,"[PAM, OLY, BAR, PAR, HTA]","[8, 4, 6, 17, 1]","[2025-11-21 00:00:00, 2025-11-26 00:00:00, 202...","[E2025_119, E2025_128, E2025_137, E2025_149, E..."
15,IST,Anadolu Efes Istanbul,7.4,5,"[BAR, MCO, MAD, PAM, ZAL]","[6, 9, 11, 8, 3]","[2025-11-20 00:00:00, 2025-11-26 00:00:00, 202...","[E2025_112, E2025_127, E2025_136, E2025_146, E..."
3,OLY,Olympiacos Piraeus,7.6,5,"[PRS, RED, ULK, BAR, PAM]","[12, 2, 10, 6, 8]","[2025-11-21 00:00:00, 2025-11-26 00:00:00, 202...","[E2025_117, E2025_128, E2025_134, E2025_148, E..."
7,PAM,Valencia Basket,8.4,5,"[RED, MUN, PAN, IST, OLY]","[2, 15, 5, 16, 4]","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_119, E2025_124, E2025_139, E2025_146, E..."
18,BAS,Baskonia Vitoria-Gasteiz,9.0,5,"[MUN, ZAL, MIL, MAD, MCO]","[15, 3, 7, 11, 9]","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_116, E2025_125, E2025_138, E2025_145, E..."
16,PAR,Partizan Mozzart Bet Belgrade,9.2,5,"[ULK, PAN, MUN, RED, VIR]","[10, 5, 15, 2, 14]","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_118, E2025_123, E2025_132, E2025_149, E..."
10,MAD,Real Madrid,9.2,5,"[ZAL, HTA, IST, BAS, MIL]","[3, 1, 16, 19, 7]","[2025-11-20 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_115, E2025_122, E2025_136, E2025_145, E..."
9,ULK,Fenerbahce Beko Istanbul,9.8,5,"[PAR, VIR, OLY, MCO, PAN]","[17, 14, 4, 9, 5]","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_118, E2025_126, E2025_134, E2025_147, E..."
19,ASV,LDLC ASVEL Villeurbanne,9.8,5,"[MCO, BAR, HTA, TEL, MUN]","[9, 6, 1, 18, 15]","[2025-11-19 00:00:00, 2025-11-26 00:00:00, 202...","[E2025_111, E2025_129, E2025_133, E2025_141, E..."


#### SOS Rest of the season

In [16]:
sos_df = compute_strength_of_schedule(standings_df, future_games)
sos_df

,teamCode,teamName,SOS,gamesRemaining,opponents,opponentRankings,gameDates,gameCodes
18,BAS,Baskonia Vitoria-Gasteiz,9.629630,27,"[MUN, ZAL, MIL, MAD, MCO, BAR, PAM, ULK, MUN, ...","[15, 3, 7, 11, 9, 6, 8, 10, 15, 20, 16, 5, 10,...","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_116, E2025_125, E2025_138, E2025_145, E..."
15,IST,Anadolu Efes Istanbul,9.851852,27,"[BAR, MCO, MAD, PAM, ZAL, DUB, ASV, RED, PRS, ...","[6, 9, 11, 8, 3, 13, 20, 2, 12, 7, 19, 1, 4, 1...","[2025-11-20 00:00:00, 2025-11-26 00:00:00, 202...","[E2025_112, E2025_127, E2025_136, E2025_146, E..."
11,PRS,Paris Basketball,9.851852,27,"[OLY, DUB, MCO, ZAL, BAR, MAD, RED, ASV, IST, ...","[4, 13, 9, 3, 6, 11, 2, 20, 16, 1, 9, 8, 13, 1...","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_117, E2025_121, E2025_131, E2025_144, E..."
4,PAN,Panathinaikos AKTOR Athens,9.925926,27,"[DUB, PAR, PAM, MIL, ULK, HTA, ZAL, OLY, MIL, ...","[13, 17, 8, 7, 10, 1, 3, 4, 7, 14, 15, 19, 18,...","[2025-11-20 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_114, E2025_123, E2025_139, E2025_143, E..."
16,PAR,Partizan Mozzart Bet Belgrade,9.925926,27,"[ULK, PAN, MUN, RED, VIR, ZAL, TEL, PAM, MCO, ...","[10, 5, 15, 2, 14, 3, 18, 8, 9, 6, 4, 15, 1, 7...","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_118, E2025_123, E2025_132, E2025_149, E..."
13,VIR,Virtus Bologna,9.925926,27,"[TEL, ULK, DUB, HTA, PAR, RED, OLY, MIL, ZAL, ...","[18, 10, 13, 1, 17, 2, 4, 7, 3, 5, 13, 10, 2, ...","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_120, E2025_126, E2025_140, E2025_150, E..."
19,ASV,LDLC ASVEL Villeurbanne,10.037037,27,"[MCO, BAR, HTA, TEL, MUN, OLY, IST, PRS, MAD, ...","[9, 6, 1, 18, 15, 4, 16, 12, 11, 19, 1, 3, 6, ...","[2025-11-19 00:00:00, 2025-11-26 00:00:00, 202...","[E2025_111, E2025_129, E2025_133, E2025_141, E..."
10,MAD,Real Madrid,10.333333,27,"[ZAL, HTA, IST, BAS, MIL, PRS, MCO, DUB, ASV, ...","[3, 1, 16, 19, 7, 12, 9, 13, 20, 18, 6, 7, 9, ...","[2025-11-20 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_115, E2025_122, E2025_136, E2025_145, E..."
9,ULK,Fenerbahce Beko Istanbul,10.407407,27,"[PAR, VIR, OLY, MCO, PAN, MIL, BAR, BAS, OLY, ...","[17, 14, 4, 9, 5, 7, 6, 19, 4, 13, 8, 14, 19, ...","[2025-11-21 00:00:00, 2025-11-25 00:00:00, 202...","[E2025_118, E2025_126, E2025_134, E2025_147, E..."
3,OLY,Olympiacos Piraeus,10.481481,27,"[PRS, RED, ULK, BAR, PAM, ASV, VIR, PAN, ULK, ...","[12, 2, 10, 6, 8, 20, 14, 5, 10, 15, 17, 18, 1...","[2025-11-21 00:00:00, 2025-11-26 00:00:00, 202...","[E2025_117, E2025_128, E2025_134, E2025_148, E..."


In [17]:
import requests
import xmltodict
import pandas as pd
import json

# --- SETTINGS ---
SEASON_CODE = "E2025"

# --- FETCH FUTURE GAMES ---
schedule_url = "https://api-live.euroleague.net/v1/schedules"
params = {"seasonCode": SEASON_CODE}

response = requests.get(schedule_url, params=params, timeout=60, verify=False)
response.raise_for_status()

data = xmltodict.parse(response.content)
games_df = pd.DataFrame(data["schedule"]["item"])

# Normalize columns
games_df["played"] = games_df["played"].astype(str).str.lower() == "true"
games_df["date"] = pd.to_datetime(games_df["date"], errors="coerce")

# Keep only unplayed games (future games)
future_games_df = games_df[~games_df["played"]].copy()
future_games_df = future_games_df.sort_values("date")

# Save to JSON
future_games_df.to_json("future_games.json", orient="records", date_format="iso")
print("Saved future_games.json")

# --- FETCH STANDINGS ---
standings_url = "https://api-live.euroleague.net/v1/standings"
response = requests.get(standings_url, params=params, timeout=60, verify=False)
response.raise_for_status()

data = xmltodict.parse(response.content)
standings_root = data["standings"]
group = standings_root["group"]
teams = group["team"]

rows = []
for t in teams:
    rows.append({
        "group": group["@name"],
        "round": group["@round"],
        "gameNumber": int(group["@gamenumber"]),
        "teamName": t["name"],
        "teamCode": t["code"],
        "ranking": int(t["ranking"]),
        "totalGames": int(t["totalgames"]),
        "wins": int(t["wins"]),
        "losses": int(t["losses"]),
        "pointsFavour": int(t["ptsfavour"]),
        "pointsAgainst": int(t["ptsagainst"]),
        "difference": int(t["difference"]),
    })

standings_df = pd.DataFrame(rows).sort_values("ranking")
standings_df.to_json("standings.json", orient="records")
print("Saved standings.json")


Saved future_games.json
Saved standings.json
